In [ ]:
!pip install mxnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


load the model from its saved state: MXNet calls this a checkpoint. In return, we get the input Symbol and the model parameters.

In [ ]:
import mxnet as mx

sym, arg_params, aux_params = mx.model.load_checkpoint('', 0)

MXNetError: ignored

create a new Module and assign it the input Symbol. We could also a context parameter indicating where we want to run the model: the default value is cpu(0), but we’d use gpu(0) to run this on a GPU.

In [ ]:
mod = mx.mod.Module(/content/Inception-BN-symbol.json)

SyntaxError: ignored

bind the input Symbol to input data. We’ll call it ‘data’ because that’s its name in the input layer of the network (look at the first few lines of the JSON file).
define the shape of ‘data’ as 1 x 3 x 224 x 224. Don’t panic ;) ‘224 x 224’ is the image resolution, that’s how the model was trained. ‘3’ is the number of channels : red, green and blue (in this order). ‘1’ is the batch size: we’ll predict one image at a time.

In [ ]:
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])

set the model parameters.

In [ ]:
mod.set_params(arg_params, aux_params)

## Preparing our data

read the image

In [ ]:
import cv2
import numpy as np
img = cv2.imread("original.jpg")

convert the image to RGB.


In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

resize the image to 224 x 224.


In [ ]:
img = cv2.resize(img, (224, 224,))

reshape the array from (image height, image width, 3) to (3, image height, image width).


In [ ]:
img = np.swapaxes(img, 0, 2)
img = np.swapaxes(img, 1, 2)

add a fourth dimension and build the NDArray


In [ ]:
img = img[np.newaxis, :]
array = mx.nd.array(img)

## Prediction

to predict a single image, so although we could use data iterator, it’d probably be overkill. Instead, we’re going to create a named tuple, called Batch, which will act as a fake iterator by returning our input NDArray when its data attribute is referenced

In [ ]:
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

Now we can pass this “batch” to the model and let it predict.


In [ ]:
mod.forward(Batch([array]))

The model will output an NDArray holding the 1000 probabilities, corresponding to the 1000 categories. It has only one line since batch size is equal to 1.


In [ ]:
prob = mod.get_outputs()[0].asnumpy()

Let’s turn this into an array with squeeze(). Then, using argsort(), we’re creating a second array holding the index of these probabilities sorted in descending order.


In [ ]:
prob = np.squeeze(prob)
print(prob.shape)
print(prob)

According to the model, the most likely category for this picture is #546 , with a probability of 58%.


In [ ]:
sortedprob = np.argsort(prob)[::-1]
sortedprob.shape
print(sortedprob)
print(prob[920])

Let’s find the name of this category. Using the synset.txt file, we can build a list of categories and find the one at index 546.


In [ ]:
synsetfile = open('synset.txt', 'r')
categorylist = []
for line in synsetfile:
  categorylist.append(line.rstrip())
print(categorylist[920])
print(categorylist[704])
print(categorylist[523])
print(categorylist[497])
print(categorylist[880])
print(categorylist[652])